# RAG dengan BM25 dan Vector Search untuk Bahasa Indonesia

Notebook ini mendemonstrasikan bagaimana membangun sistem Retrieval Augmented Generation (RAG) untuk Bahasa Indonesia menggunakan kombinasi dari BM25 (sebuah algoritma pencarian berbasis kata kunci) dan pencarian vektor (berbasis semantic similarity). Kita akan menggunakan LangChain untuk mengorkestrasi komponen-komponen ini.

## Setup

Instalasi library yang dibutuhkan. Jika Anda menggunakan Google Colab, beberapa library mungkin sudah terinstal. Pastikan juga Anda memiliki API Key untuk LLM yang akan digunakan (misalnya OpenAI) dan telah mengaturnya sebagai environment variable.

In [ ]:
!pip install -qU langchain langchain-community rank_bm25 sentence-transformers faiss-cpu openai tiktoken

Anda mungkin perlu mengatur environment variable untuk API Key OpenAI. Contoh:
```python
import os
os.environ['OPENAI_API_KEY'] = 'SKOR_API_KEY_ANDA'
```
Pastikan untuk mengganti `'SKOR_API_KEY_ANDA'` dengan API key OpenAI Anda yang sebenarnya jika menggunakan model OpenAI.

## Data Preparation

Kita akan membuat beberapa contoh dokumen dalam Bahasa Indonesia. Setiap dokumen akan memiliki ID unik dan teks konten.

In [ ]:
sample_documents_data = [
    {"id": "doc1", "text": "Sungai Kapuas adalah sungai terpanjang di Indonesia."}, 
    {"id": "doc2", "text": "Candi Borobudur merupakan candi Buddha terbesar di dunia dan terletak di Magelang, Jawa Tengah."},
    {"id": "doc3", "text": "Rendang adalah masakan daging dengan bumbu rempah-rempah yang berasal dari Minangkabau, Sumatera Barat."},
    {"id": "doc4", "text": "Gunung Bromo adalah gunung berapi aktif yang terkenal dengan pemandangan matahari terbitnya."},
    {"id": "doc5", "text": "Bahasa Indonesia adalah bahasa resmi Republik Indonesia dan bahasa persatuan bangsa Indonesia."},
    {"id": "doc6", "text": "Danau Toba adalah danau vulkanik terbesar di dunia yang terletak di Sumatera Utara."},
    {"id": "doc7", "text": "Tari Saman dari Aceh dikenal dengan kecepatan dan kekompakan gerakannya."}
]

print(f"Jumlah dokumen: {len(sample_documents_data)}")
print(f"Contoh dokumen pertama: {sample_documents_data[0]}")

Selanjutnya, kita ubah data mentah ini menjadi objek `Document` dari LangChain. Kita akan menggunakan 'text' sebagai `page_content` dan 'id' sebagai `metadata`.

In [ ]:
from langchain_core.documents import Document

langchain_documents = []
for doc_data in sample_documents_data:
    langchain_documents.append(Document(page_content=doc_data['text'], metadata={'id': doc_data['id']}))

print(f"Jumlah dokumen LangChain: {len(langchain_documents)}")
print(f"Contoh dokumen LangChain pertama: {langchain_documents[0]}")

## BM25 Retriever

BM25 adalah algoritma ranking berbasis kata kunci yang efektif. Kita akan menggunakan implementasi `BM25Okapi` dari library `rank_bm25` dan membungkusnya dengan `BM25Retriever` dari LangChain.

Penting: BM25 bekerja dengan teks yang sudah di-tokenize. Untuk Bahasa Indonesia, idealnya kita menggunakan tokenizer khusus Bahasa Indonesia. Namun, untuk kesederhanaan, contoh ini akan melakukan split berdasarkan spasi.

In [ ]:
from langchain.retrievers import BM25Retriever

# Inisialisasi BM25Retriever dengan dokumen kita
# Kita bisa langsung memasukkan Document LangChain
bm25_retriever = BM25Retriever.from_documents(langchain_documents)
bm25_retriever.k = 3 # Ambil top 3 dokumen

# Contoh penggunaan
query_bm25 = "candi di Jawa Tengah"
retrieved_docs_bm25 = bm25_retriever.invoke(query_bm25)

print(f"Query BM25: {query_bm25}")
print("Dokumen yang diambil oleh BM25:")
for doc in retrieved_docs_bm25:
    print(f"- ID: {doc.metadata['id']}, Teks: {doc.page_content}")

## Vector Retriever (menggunakan model multilingual)

Untuk pencarian semantik, kita akan menggunakan embedding dan vector store. Kita akan menggunakan `HuggingFaceEmbeddings` dengan model `sentence-transformers/paraphrase-multilingual-mpnet-base-v2` yang mendukung Bahasa Indonesia, dan FAISS sebagai vector store.

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Inisialisasi model embedding
embedding_model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

# Buat vector store FAISS dari dokumen dan embeddings
try:
    vector_store = FAISS.from_documents(langchain_documents, embeddings)
except IndexError as e:
    print(f"Error saat membuat FAISS index: {e}. Ini kadang terjadi jika hanya ada 1 dokumen. Coba tambahkan lebih banyak dokumen.")
    # Handle error atau stop eksekusi jika perlu
    vector_store = None # Atau inisialisasi dengan cara lain jika memungkinkan

vector_retriever = None
if vector_store:
    # Buat retriever dari vector store
    vector_retriever = vector_store.as_retriever(search_kwargs={"k": 3})

    # Contoh penggunaan
    query_vector = "tempat ibadah bersejarah di Jawa"
    retrieved_docs_vector = vector_retriever.invoke(query_vector)

    print(f"\nQuery Vector: {query_vector}")
    print("Dokumen yang diambil oleh Vector Retriever:")
    for doc in retrieved_docs_vector:
        print(f"- ID: {doc.metadata['id']}, Teks: {doc.page_content}")
else:
    print("\nVector store tidak berhasil diinisialisasi, Vector Retriever tidak akan dijalankan.")

## Ensemble Retriever

Menggabungkan kekuatan BM25 (pencarian kata kunci) dan Vector Retriever (pencarian semantik) menggunakan `EnsembleRetriever`. Ini seringkali memberikan hasil yang lebih baik daripada menggunakan salah satu retriever saja.

Kita akan memberikan bobot pada masing-masing retriever. Bobot ini menentukan seberapa besar kontribusi masing-masing retriever pada skor akhir dokumen.

In [ ]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = None
# Pastikan kedua retriever sudah diinisialisasi
if bm25_retriever and vector_retriever:
    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, vector_retriever],
        weights=[0.5, 0.5]  # Bobot bisa disesuaikan
    )

    # Contoh penggunaan
    query_ensemble = "objek wisata populer di Indonesia"
    retrieved_docs_ensemble = ensemble_retriever.invoke(query_ensemble)

    print(f"\nQuery Ensemble: {query_ensemble}")
    print("Dokumen yang diambil oleh Ensemble Retriever:")
    for doc in retrieved_docs_ensemble:
        print(f"- ID: {doc.metadata['id']}, Teks: {doc.page_content}")
else:
    print("\nSalah satu atau kedua retriever (BM25 atau Vector) tidak berhasil diinisialisasi. Ensemble Retriever tidak akan dijalankan.")

## RAG Chain (Question Answering)

Sekarang kita akan membangun RAG chain untuk menjawab pertanyaan berdasarkan dokumen yang diambil oleh retriever (kita akan menggunakan `ensemble_retriever` jika tersedia, jika tidak, fallback ke `bm25_retriever` atau `vector_retriever`).

**Penting:** Bagian ini menggunakan `ChatOpenAI`. Anda **harus** memiliki API Key OpenAI dan telah mengaturnya sebagai environment variable `OPENAI_API_KEY` agar kode di bawah ini berjalan. Jika tidak, Anda bisa menggantinya dengan LLM lain yang kompatibel dengan LangChain.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Pilih retriever yang akan digunakan
final_retriever = None
if ensemble_retriever:
    print("Menggunakan Ensemble Retriever untuk RAG chain.")
    final_retriever = ensemble_retriever
elif vector_retriever:
    print("Ensemble retriever tidak tersedia. Menggunakan Vector Retriever untuk RAG chain.")
    final_retriever = vector_retriever
elif bm25_retriever:
    print("Ensemble dan Vector retriever tidak tersedia. Menggunakan BM25 Retriever untuk RAG chain.")
    final_retriever = bm25_retriever
else:
    print("Tidak ada retriever yang tersedia. RAG chain tidak dapat dibuat.")

rag_chain = None
if final_retriever:
    # Template prompt
    template = """Anda adalah asisten AI yang membantu menjawab pertanyaan berdasarkan konteks yang diberikan.
    Gunakan hanya informasi dari konteks berikut untuk menjawab pertanyaan.
    Jika Anda tidak tahu jawabannya berdasarkan konteks, katakan saja Anda tidak tahu.
    Jangan mencoba membuat jawaban.
    Jawablah dalam Bahasa Indonesia.

    Konteks:
    {context}

    Pertanyaan: {question}

    Jawaban:
    """
    prompt = ChatPromptTemplate.from_template(template)

    # Inisialisasi LLM (pastikan OPENAI_API_KEY sudah di-set)
    try:
        # Coba import os untuk memeriksa environment variable
        import os
        if not os.getenv('OPENAI_API_KEY'):
            print("Peringatan: Environment variable OPENAI_API_KEY tidak diatur.")
            print("RAG chain dengan OpenAI tidak akan berfungsi tanpa API Key.")
            # Anda bisa memilih untuk raise error di sini atau membiarkan ChatOpenAI gagal saat dipanggil
        llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
    except Exception as e:
        print(f"Error saat inisialisasi ChatOpenAI: {e}")
        print("Pastikan Anda telah mengatur environment variable OPENAI_API_KEY jika menggunakan OpenAI.")
        print("Anda dapat mengganti ChatOpenAI dengan LLM lain jika diperlukan.")
        llm = None

    if llm:
        # RAG Chain
        rag_chain = (
            {"context": final_retriever | format_docs, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

        # Contoh pertanyaan
        question_rag = "Di mana letak Candi Borobudur?"
        print(f"\nPertanyaan RAG: {question_rag}")

        try:
            # Hanya jalankan invoke jika OPENAI_API_KEY ada
            if os.getenv('OPENAI_API_KEY'):
                answer = rag_chain.invoke(question_rag)
                print(f"Jawaban RAG: {answer}")
            else:
                print("Tidak menjalankan RAG chain karena OPENAI_API_KEY tidak diatur.")
        except Exception as e:
            print(f"Error saat menjalankan RAG chain: {e}")
            if "OPENAI_API_KEY" in str(e) or "api_key" in str(e).lower():
                print("Pastikan environment variable OPENAI_API_KEY sudah benar dan memiliki kredit yang cukup.")

        question_rag_2 = "Apa itu Rendang?"
        print(f"\nPertanyaan RAG 2: {question_rag_2}")
        try:
            if os.getenv('OPENAI_API_KEY'):
                answer_2 = rag_chain.invoke(question_rag_2)
                print(f"Jawaban RAG 2: {answer_2}")
            else:
                print("Tidak menjalankan RAG chain karena OPENAI_API_KEY tidak diatur.")
        except Exception as e:
            print(f"Error saat menjalankan RAG chain: {e}")
    else:
        print("LLM tidak berhasil diinisialisasi. RAG chain tidak akan dijalankan.")
else:
    print("RAG chain tidak dijalankan karena tidak ada retriever yang valid.")

## Kesimpulan

Notebook ini telah mendemonstrasikan langkah-langkah untuk membangun sebuah sistem RAG sederhana untuk Bahasa Indonesia. Kita telah mencakup:
1.  Persiapan data contoh dalam Bahasa Indonesia.
2.  Penggunaan `BM25Retriever` untuk pencarian berbasis kata kunci.
3.  Penggunaan `VectorRetriever` dengan embedding multilingual (`paraphrase-multilingual-mpnet-base-v2`) untuk pencarian semantik.
4.  Penggabungan kedua retriever menggunakan `EnsembleRetriever` untuk mendapatkan hasil yang lebih relevan.
5.  Pembuatan RAG chain dengan LangChain Expression Language (LCEL) untuk menjawab pertanyaan berdasarkan konteks yang diambil, dengan catatan penting mengenai penggunaan LLM dan API Key.

Pendekatan hybrid dengan ensemble retriever seringkali memberikan hasil yang lebih robas, menggabungkan keunggulan pencarian leksikal dan semantik. Untuk aplikasi nyata, Anda mungkin perlu menggunakan dataset yang lebih besar, tokenizer Bahasa Indonesia yang lebih canggih, dan LLM yang lebih mumpuni, serta melakukan evaluasi yang komprehensif.

--- 
Terima kasih telah mengikuti notebook ini! Semoga bermanfaat.